In [7]:


class For_Model():
    
    def __init__(self):
        self.mecab = MeCab.Tagger("-Owakati")
        self.data = './output/buzz_tweet.csv' #取得したデータのパス
        self.columns = ["Followers", "Full_text","Posi_score", "Nega_score","Judge"] #取得したい列名
        self.out_file = "train_buzz.txt" #出力ファイル名
        self.mode = 'w' #学習データの保存モード　'a'：追加／'w'：上書き
        self.text = "Full_text" #ツイートテキストの列を指定
        similar = str(input("ツイート予定文書を入力してください\n"))
        self.similar = str(similar)

    def Stop_Words(self):
        # ストップワードをダウンロード
        url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
        urllib.request.urlretrieve(url, './output/stop_word.txt')

        with open('./output/stop_word.txt', 'r', encoding='utf-8') as file:
            stopwords = [word.replace('\n', '') for word in file.readlines()]

        #追加ストップワードを設定（助詞や意味のない平仮名１文字）
        add_words = ['あ','い','う','え','お','か','き','く','け','こ','さ','し','す','せ','そ','た','ち','つ','て','と',
                     'な','に','ぬ','ね','の','は','ひ','ふ','へ','ほ','ま','み','む','め','も','や','ゆ','よ',
                     'ら','り','る','れ','ろ','わ','を','ん','が','ぎ','ぐ','げ','ご','ざ','じ','ず','ぜ','ぞ',
                     'だ','ぢ','づ','で','ど','ば','び','ぶ','べ','ぼ','ぱ','ぴ','ぷ','ぺ','ぽ',
                     'くん','です','ます','ました','そして','でも','だから','だが','くらい','その','それ','かも',
                     'あれ','あの','あっ','そんな','この','これ','とか','とも','する','という','ござい',
                     'ので','なんて','たら', 'られ','たい','さて','てる','ください','なる','けど','でし',
                     'じゃん','だっ','なっ','でしょ', 'ある','って','こんな','ねえ'
                    ]
        stopwords = stopwords + add_words
        return stopwords

    def Tokenizer(self, text, stopwords):

        words = []
        text = self.mecab.parse(text)
        text = text.split(' ')
        for j in range(len(text)):
            if text[j] not in stopwords:
                words.append(text[j])
        return words



    #記号削除
    def format_text(self, text):
        text = unicodedata.normalize("NFKC", text)  # 全角記号を半角へ置換
        # 記号を消し去るための魔法のテーブル作成
        table = str.maketrans("", "", string.punctuation  + "「」、。・*`+-|?#!()\[]<>=~/")
        text = text.translate(table)
        return text

    def main(self):
        tweets_num = 0
        stopwords = self.Stop_Words()
        df_tweet, tweets = self.Load_tweets()
        #ツイートを分かち書きしてcsvに出力(モード'a'はデータ追加、モード'w'は新規作成)
        with open('./output/' + self.out_file, self.mode) as f:
            for i in tweets:
                tweets_num += 1
                i = re.sub('\n', "", i)
                i = re.sub(r'[!-~]', "", i)#半角記号,数字,英字を削除
                i = re.sub(r'[︰-＠]', "", i)#全角記号削除
                i = self.format_text(i)#記号削除
                i = re.sub(r'[【】●ㅅ●Ф☆✩︎♡→←▼①②③④⑤『』ω《》∠∇∩♪∀◞ཀCщ≧≦ ́◤◢■◆★※↑↓〇◯○◎⇒▽◉Θ♫♬〃“”◇ᄉ⊂⊃д°]', "", i)
                #i = re.sub(r'[‥…？！〜「」｢｣:：♪♩『』→↓↑〈〉・゜・´Д´°ω°•ω•★＊☆♡（）✔Θ∀´∀｀˘ω˘‼бωб￣▽￣]', "", i)
                i = self.remove_emoji(i)
                i = self.Tokenizer(i, stopwords)
                i = ' '.join(i) #リストを文字列に変換
                i = str(i)
                f.write(i)

        with open('./output/' + self.out_file) as f:
             wakati = f.read()

        print('csv出力完了：'+ self.out_file)
        print("学習用ツイート数（判定用ツイート含む/短すぎるツイートは削除）：", tweets_num)
        print("[分かち書きサンプル]\n", wakati[:100])
        print()
        return df_tweet, self.similar

In [107]:
import MeCab
import re
import pandas as pd
import pprint
import emoji
import urllib.request
import unicodedata
import string

class JpTextsPreProcess():
    def __init__(self):
        self.mecab = MeCab.Tagger("-Owakati")
        
    def stop_words(self):
            # ストップワードをダウンロード
            url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
            urllib.request.urlretrieve(url, './output/stop_word.txt')

            with open('./output/stop_word.txt', 'r', encoding='utf-8') as file:
                stopwords = [word.replace('\n', '') for word in file.readlines()]

            #追加ストップワードを設定（助詞や意味のない平仮名１文字）
            add_words = ['あ','い','う','え','お','か','き','く','け','こ','さ','し','す','せ','そ','た','ち','つ','て','と',
                         'な','に','ぬ','ね','の','は','ひ','ふ','へ','ほ','ま','み','む','め','も','や','ゆ','よ',
                         'ら','り','る','れ','ろ','わ','を','ん','が','ぎ','ぐ','げ','ご','ざ','じ','ず','ぜ','ぞ',
                         'だ','ぢ','づ','で','ど','ば','び','ぶ','べ','ぼ','ぱ','ぴ','ぷ','ぺ','ぽ',
                         'くん','です','ます','ました','そして','でも','だから','だが','くらい','その','それ','かも',
                         'あれ','あの','あっ','そんな','この','これ','とか','とも','する','という','ござい',
                         'ので','なんて','たら', 'られ','たい','さて','てる','ください','なる','けど','でし',
                         'じゃん','だっ','なっ','でしょ', 'ある','って','こんな','ねえ', "\n"
                        ]
            stopwords = stopwords + add_words
            return stopwords

    def remove_emoji(self, text):
        return ''.join(c for c in text if c not in emoji.UNICODE_EMOJI)


    def format_text(self, text):
        text = unicodedata.normalize("NFKC", text)  # 全角記号を半角へ置換
        # 記号を消し去るための魔法のテーブル作成
        table = str.maketrans("", "", string.punctuation  + "「」、。・*`+-|?#!()\[]<>=~/")
        text = text.translate(table)
        return text

    def Tokenizer(self, text, stopwords):
        words = []
        text = self.mecab.parse(text)
        text = text.split(' ')
        for j in range(len(text)):
            if text[j] not in stopwords:
                words.append(text[j])
        return words

    def preprocess(self, text):
        stopwords = self.stop_words()
        text = text.lower()
        text = re.sub('\n', "", text) #改行削除

        text = re.sub(r'[︰-＠]', "", text)#全角記号削除
        text = self.format_text(text)#記号削除
        text = re.sub(r'[【】●ㅅ●Ф☆✩︎♡→←▼①②③④⑤『』ω《》∠∇∩♪∀◞ཀCщ≧≦ ́◤◢■◆★※↑↓〇◯○◎⇒▽◉Θ♫♬〃“”◇ᄉ⊂⊃д°]', "", text)
        text = re.sub(r'[‥…？！〜「」｢｣:：♪♩『』→↓↑〈〉・゜・´Д´°ω°•ω•★＊☆♡（）✔Θ∀´∀｀˘ω˘‼бωб￣▽￣]', "", text)
        text = self.remove_emoji(text)
        text = self.Tokenizer(text, stopwords)
        text = ' '.join(text) #リストを文字列に変換
        text = str(text)
        return text


In [108]:
#df = pd.read_pickle("data_20201009.pkl")

In [110]:
preprocess = JpTextsPreProcess()


In [122]:
df["title"][350:368].apply(preprocess.preprocess)

412                                  irobot ブラーバ 371 j
413                                        ヤス ヤス さま 専用
414                bluetooth イヤホン 音質 shurese 215 ワイヤレス
415                    bluetooth ノイズ キャン セリング ヘッドホン 新品
416                      サン ディスク sd カード sandisk 128 gb
417         スラム ダンク slamdunkiphone ケース iphone 7 ジャンプ 展
418                即納 wahlshavershaper ウォール バリカン シェーバー
419              hviewptz 防犯 カメラ 5 x 光学 ズーム 500 画素 poe
420                                  oculusquest 64 gb
421                     ロータス 専用 sonynex − 5 nnex 5 ndb
422                    最新 50 騒音 遮断 音質 ️ bluetooth イヤホン
423                                リンナイガスファンヒーター 都市 ガス
424                  ipadpro 97 wifi 32 gb 画面 割れ あり 動作
425                    ファー ウェイ huaweinovaliteforymobil
426         sony ‪ α ‬ a マウント 望遠 ズーム レンズ 70300 mmf 456
427                      iodataexld 2381 db 238 型 モニター
428    学習 コンテンツ 豊富 高校 生 モデル 大学 受験 センター 試験 対策 受験 特化 モデル
429          n i k o n c oolpixperformancecoolpixp 310
Name: titl

In [124]:
preprocess.preprocess(df["title"][368])

'iphone 6 シルバー 64 gb 値段 交渉 ️'

In [135]:
"".join(emoji.UNICODE_EMOJI.keys())

'🥇🥈🥉🆎🏧🅰🅰️🇦🇫🇦🇽🇦🇱🇩🇿🇦🇸🇦🇩🇦🇴🇦🇮🇦🇶🇦🇬♒🇦🇷♈🇦🇲🇦🇼🇦🇨🇦🇺🇦🇹🇦🇿🔙🅱🅱️🇧🇸🇧🇭🇧🇩🇧🇧🇧🇾🇧🇪🇧🇿🇧🇯🇧🇲🇧🇹🇧🇴🇧🇦🇧🇼🇧🇻🇧🇷🇮🇴🇻🇬🇧🇳🇧🇬🇧🇫🇧🇮🆑🆒🇰🇭🇨🇲🇨🇦🇮🇨♋🇨🇻♑🇧🇶🇰🇾🇨🇫🇪🇦🇹🇩🇨🇱🇨🇳🇨🇽🎄🇨🇵🇨🇨🇨🇴🇰🇲🇨🇬🇨🇩🇨🇰🇨🇷🇭🇷🇨🇺🇨🇼🇨🇾🇨🇿🇨🇮🇩🇰🇩🇬🇩🇯🇩🇲🇩🇴🔚🇪🇨🇪🇬🇸🇻🏴\U000e0067\U000e0062\U000e0065\U000e006e\U000e0067\U000e007f🇬🇶🇪🇷🇪🇪🇪🇹🇪🇺🆓🇫🇰🇫🇴🇫🇯🇫🇮🇫🇷🇬🇫🇵🇫🇹🇫🇬🇦🇬🇲♊🇬🇪🇩🇪🇬🇭🇬🇮🇬🇷🇬🇱🇬🇩🇬🇵🇬🇺🇬🇹🇬🇬🇬🇼🇬🇳🇬🇾🇭🇹🇭🇲🇭🇳🇭🇰🇭🇺🆔🇮🇸🇮🇳🇮🇩🇮🇷🇮🇶🇮🇪🇮🇲🇮🇱🇮🇹🇯🇲🇯🇵🉑🈸🉐🏯㊗🈹🎎🈚🈁🈷🈵🈶🈺🈴🏣🈲🈯㊙🈂🔰🈳㊗️🈷️㊙️🈂️🇯🇪🇯🇴🇰🇿🇰🇪🇰🇮🇽🇰🇰🇼🇰🇬🇱🇦🇱🇻🇱🇧♌🇱🇸🇱🇷♎🇱🇾🇱🇮🇱🇹🇱🇺🇲🇴🇲🇰🇲🇬🇲🇼🇲🇾🇲🇻🇲🇱🇲🇹🇲🇭🇲🇶🇲🇷🇲🇺🇾🇹🇲🇽🇫🇲🇲🇩🇲🇨🇲🇳🇲🇪🇲🇸🇲🇦🇲🇿🤶🤶🏿🤶🏻🤶🏾🤶🏼🤶🏽🇲🇲🆕🆖🇳🇦🇳🇷🇳🇵🇳🇱🇳🇨🇳🇿🇳🇮🇳🇪🇳🇬🇳🇺🇳🇫🇰🇵🇲🇵🇳🇴🆗👌👌🏿👌🏻👌🏾👌🏼👌🏽🔛🅾🅾️🇴🇲⛎🅿🅿️🇵🇰🇵🇼🇵🇸🇵🇦🇵🇬🇵🇾🇵🇪🇵🇭♓🇵🇳🇵🇱🇵🇹🇵🇷🇶🇦🇷🇴🇷🇺🇷🇼🇷🇪🔜🆘♐🇼🇸🇸🇲🎅🎅🏿🎅🏻🎅🏾🎅🏼🎅🏽🇸🇦♏🏴\U000e0067\U000e0062\U000e0073\U000e0063\U000e0074\U000e007f🇸🇳🇷🇸🇸🇨🇸🇱🇸🇬🇸🇽🇸🇰🇸🇮🇸🇧🇸🇴🇿🇦🇬🇸🇰🇷🇸🇸🇪🇸🇱🇰🇧🇱🇸🇭🇰🇳🇱🇨🇲🇫🇵🇲🇻🇨🗽🇸🇩🇸🇷🇸🇯🇸🇿🇸🇪🇨🇭🇸🇾🇸🇹\U0001f996🔝🇹🇼🇹🇯🇹🇿♉🇹🇭🇹🇱🇹🇬🇹🇰🗼🇹🇴🇹🇹🇹🇦🇹🇳🇹🇷🇹🇲🇹🇨🇹🇻🇺🇲🇻🇮🆙🇺🇬🇺🇦🇦🇪🇬🇧🇺🇳🇺🇸🇺🇾🇺🇿🆚🇻🇺🇻🇦🇻🇪🇻🇳♍🏴\U000e0067\U000e0062\U000e0077\U000e006c\U000e0073\U000e007f🇼🇫🇪🇭🇾🇪🇿🇲🇿🇼\U0001f9ee\U0001fa79🎟🎟️\U0001f9d1\U0001f9d1🏿\U0001f9d1🏻\U0001f9d1🏾\U0001f9d1🏼\U0001f9d1🏽🚡✈🛬🛫✈️⏰⚗⚗️👽👾🚑🏈🏺⚓💢😠👿😧🐜📶😰🚛🎨😲⚛⚛️\U0001f6fa🚗🥑\U0001fa93👶👼👼🏿👼🏻👼🏾👼🏼👼🏽🍼🐤👶🏿👶🏻👶🏾👶🏼👶🏽🚼👇👇🏿👇🏻👇🏾👇🏼👇🏽👈👈🏿👈🏻